In [2]:
import os 
import numpy as np
import pandas as pd 
from tqdm import tqdm
import pandas as pd
tqdm.pandas(desc='pandas bar')
# from pandarallel import pandarallel
import random
from tqdm.notebook import tqdm
import copy
import logging
# pandarallel.initialize(progress_bar=True)

import argparse
from types import SimpleNamespace
from FlagEmbedding import FlagModel

In [7]:
amazon_google_train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/ditto_CL_train.csv',index_col=0)

In [3]:
amazon_google_train_table = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/train.csv')

In [8]:
# amazon_google_train_table
amazon_google_tableA = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/tableA.csv')
amazon_google_tableB = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/tableB.csv')

In [ ]:
def Extract_Title(row):
    split_a = row[0].split('COL title VAL ')[1].split(' COL')[0]
    split_b = row[1].split('COL title VAL ')[1].split(' COL')[0]
    print(split_a)
    id_a = amazon_google_tableA[amazon_google_tableA['title']==split_a].iloc[0,0]
    id_b = amazon_google_tableB[amazon_google_tableB['title']==split_b].iloc[0,0]
    return id_a,id_b
amazon_google_train_pos = amazon_google_train[amazon_google_train['2']==1]
# amazon_google_train_pos['0'].str.split('COL title VAL ')[0].split(' COL manufacturer')[0]
amazon_google_train_pos_id = amazon_google_train_pos.progress_apply(Extract_Title,axis=1)

In [35]:
np.save('/data/home/wangys/LLM_ER/process_data/amazon-google/pos_list.npy',np.array(amazon_google_train_pos_id))

In [7]:
dict_ltable = np.load('/data/home/wangys/LLM_ER/process_data/amazon-google/dict_ltable.npy',allow_pickle=True).item()
dict_rtable = np.load('/data/home/wangys/LLM_ER/process_data/amazon-google/dict_rtable.npy',allow_pickle=True).item()

In [9]:
tableA = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/tableA.csv').fillna('')
tableB = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/tableB.csv').fillna('')
tableA_dict = tableA.set_index('id').to_dict(orient='records')
tableB_dict = tableB.set_index('id').to_dict(orient='records')
train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/train.csv')
valid = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/valid.csv')
test = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon-google/test.csv')

In [9]:
tableA_dict_update = {}
tableB_dict_update = {}
for i in range(len(tableA_dict)):
    tableA_dict_update[i] = tableA_dict[i]
for i in range(len(tableB_dict)):
    tableB_dict_update[i] = tableB_dict[i]

In [ ]:
## We try a different Negative Sampling Strategy here

In [72]:
# amazon_google_train_pos_id
query_list = []
for pos_id_pair in tqdm(amazon_google_train_pos_id):
    ltable_id = pos_id_pair[0]
    rtable_id = pos_id_pair[1]
    for l in dict_ltable[ltable_id]: ## All Dict in ltable
        query_list.append([str(l),[str(r) for r in dict_rtable[rtable_id]],[]])

  0%|          | 0/100 [00:00<?, ?it/s]

In [73]:
query_df = pd.DataFrame(query_list)
query_df.columns = ['query','pos','neg']

In [584]:
import json
df = query_list_pd[query_list_pd.iloc[:,1].astype(str)!='[]']
df.columns = ['query','pos','neg']
# 打开一个文件以写入
path = '/data/home/wangys/MoE-Example/SM/CMS/SBert/CMS-blocking-update.json'
with open(path, 'w', encoding='utf-8') as file:
    for index, row in tqdm(df.iterrows()):
        # 对于每行，创建一个字典
        data = {
            "query": row["query"],
            "pos": row["pos"],
            "neg": row["neg"]
        }

        # 将字典转换为JSON字符串，并写入文件
        file.write(json.dumps(data, ensure_ascii=False))
        file.write('\n')  # 每个JSON对象后添加换行符

print(path)

0it [00:00, ?it/s]

/data/home/wangys/MoE-Example/SM/CMS/SBert/CMS-blocking-update.json


In [ ]:
## Process the Mapping of Input and Output

In [10]:
ltable_list = []
rtable_list = []
count = 0
for key in dict_ltable.keys():
    for l in dict_ltable[key]:
        ltable_list.append([count,key,str(l)])
        count += 1
count = 0
for key in dict_rtable.keys():
    for r in dict_rtable[key]:
        rtable_list.append([count,key,str(r)])
        count += 1

In [11]:
ltable_list = pd.DataFrame(ltable_list)
rtable_list = pd.DataFrame(rtable_list)
ltable_list.columns = ['index','id','query']
rtable_list.columns = ['index','id','query']

In [12]:
ltable_index2id = dict(zip(ltable_list['index'],ltable_list['id']))
rtable_index2id = dict(zip(rtable_list['index'],rtable_list['id']))

In [13]:
from FlagEmbedding import FlagModel

In [14]:
model = FlagModel('/data/home/wangys/MoE-Example/ER/amazon-google/SBert/rerank-em', 
                  use_fp16=True)

----------using 8*GPUs----------


In [50]:
# embedding_a = model.encode(ltable_list['query'].to_list())
embedding_a = model.encode([str(tableA_dict_update[i]) for i in range(len(tableA_dict_update))])
embedding_b = model.encode(rtable_list['query'].to_list())
# similarity = embedding_a @ embedding_b.T

Inference Embeddings: 100%|██████████| 7/7 [00:03<00:00,  2.14it/s]


In [51]:
import torch
embedding_a = torch.DoubleTensor(embedding_a) # 64 bit
embedding_b = torch.DoubleTensor(embedding_b)
if (torch.cuda.is_available()):
    embedding_a = torch.DoubleTensor(embedding_a).cuda()
    embedding_b = torch.DoubleTensor(embedding_b).cuda()

In [52]:
out_array = torch.mm(embedding_a, embedding_b.T).cpu().numpy()

In [57]:
ltable_list
# def L2R_search(row): ## augment-augment
#     index = row['index']
#     r_index = np.argsort(-out_array[index])[0] ## Most Similar
#     r_id = rtable_index2id[r_index]
#     return r_id
k = 2
def L2R_search(row):
    index = row['id']
    r_index = np.argsort(-out_array[index])[0:k] ## Most Similar
    r_id = [rtable_index2id[r] for r in r_index]
    return r_id
ltable_list['r_id'] = ltable_list.progress_apply(L2R_search,axis=1)

pandas bar: 100%|██████████| 7357/7357 [00:06<00:00, 1209.20it/s]


In [33]:
amazon_all = pd.concat([train,valid,test])
amazon_all_pos = amazon_all[amazon_all['label']==1]

In [59]:
base_num = 0
recall_num = 0
for index,row in amazon_all_pos.iterrows():
    ltable_id = row['ltable_id'] ## ltable_id
    rtable_id = row['rtable_id']
    search_df = ltable_list[ltable_list['id']==ltable_id]
    voting_num = search_df['r_id'].value_counts().idxmax()
    if(rtable_id in voting_num):
        recall_num += 1
    base_num += 1
recall_num/base_num

0.7960582690659811

In [1]:
amazon_google = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon_google_train_filter.csv',index_col=0)

NameError: name 'pd' is not defined

In [60]:
amazon_google_test = pd.read_csv('/data/home/wangys/LLM_ER/process_data/amazon-google/amazon_google_test_filter.csv',index_col=0)

In [17]:
ast.literal_eval(amazon_google.iloc[0,3])

{'title': 'tlc dr. seuss reading learning system 2008',
 'manufacturer': 'encore',
 'price': 19.99,
 'category': 'Education',
 'subcategory': 'Language Learning',
 'platform': 'Windows',
 'edition': 'Standard',
 'type': 'Software',
 'modelno': 'RSLS-008'}

In [16]:
import json
import ast
temp_dict = {}
temp_dict['Output'] = ''
json.dumps(temp_dict)

'{"Output": ""}'

In [ ]:
for a in amazon_google_train_pos_id:
    print(a[0])

In [37]:
# amazon_google_train_pos_id
ind = []
for a in amazon_google_train_pos_id:
    i = amazon_google[(amazon_google['ltable_id']==a[0]) & (amazon_google['rtable_id']==a[1])].index
    # print(i)
    ind.extend(list(i))

In [24]:
amazon_google_pos = amazon_google[amazon_google['label']==1]
len(amazon_google_pos['rtable_id'].unique())

100

In [81]:
def Prompt_Generalize(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s\n\nTake these examples as reference:' % (json.dumps(temp_dict),json.dumps(ast.literal_eval(row['l_entity'])),json.dumps(ast.literal_eval(row['r_entity'])))
    select_df = amazon_google[(amazon_google['ltable_id']==ltable_id) | (amazon_google['rtable_id']==rtable_id)] ## 相同元素
    select_df = select_df[~((select_df['ltable_id']==ltable_id) & (select_df['rtable_id']==rtable_id))] ## 排除自己
    select_df_pos = select_df[select_df['label']==1]
    select_df_neg = select_df[select_df['label']==0]
    all_neg = amazon_google[amazon_google['label']==0]
    # if(len(select_df_pos)>=3): ## Contain pos
    #     for index,row_ in select_df_pos.sample(n=4,replace=False).iterrows():
    #         print(index)
    #         RAG_dict = {}
    #         RAG_dict['Entity_1'] = str(row_['l_entity'])
    #         RAG_dict['Entity_2'] = str(row_['r_entity'])
    #         RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
    #         RAG_text += '%s\n\n' % json.dumps(RAG_dict)
    # else:
    if(len(select_df_neg)>=4):
        df = pd.concat([amazon_google_gt.sample(n=4,replace=False),select_df_neg.sample(n=4,replace=False)]).sample(frac=1)
    else:
        df = pd.concat([amazon_google_gt.sample(n=4,replace=False),select_df_neg.sample(frac=1,replace=False),all_neg.sample(n=4-len(select_df_neg),replace=False)]).sample(frac=1)
    for index,row_ in df.iterrows():
        
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(row_['l_entity'])
        RAG_dict['Entity_2'] = str(row_['r_entity'])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
amazon_google_output = amazon_google.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 6612/6612 [00:16<00:00, 391.09it/s]


In [84]:
amazon_google_output.columns = ['instruction','input','output']

In [93]:
with open('/data/home/wangys/LLaMA-Factory-main/lora_weight/MoE/ER/llama-2-chat/amazon-google-train-MoE/adapter_config.json') as f:
  data = json.load(f)


In [309]:
json.dump(ant_buy_test_output.to_dict(orient='records'), open('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/ant_buy_test_output.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [527]:
df = CMS_test_output.iloc[:,:3]

df.columns = ['instruction','input','output']
json.dump(df.to_dict(orient='records'), open('/data/home/wangys/LLaMA-Factory-main/data/MoE/SM/synthea_test_output.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [2]:
import hashlib


def encrypt(fpath: str, algorithm: str) -> str:
    with open(fpath, 'rb') as f:
        return hashlib.new(algorithm, f.read()).hexdigest()


if __name__ == '__main__':
    for algorithm in ('md5', 'sha1', 'sha256'):
        hexdigest = encrypt('/data/home/wangys/LLaMA-Factory-main/data/MoE/DI/amazon_train_all.json', algorithm)
        print(f'{algorithm}: {hexdigest}')

md5: 6caf0ae3f121b4fb67b9a6287ced5973
sha1: 3e88ea7a6940b48cc3694be60c1bdb8990be6663
sha256: dd44e169c9f8ccfb211d49d68f767b6cafef1a34af8a90b2b4447a9dbad270e0


In [ ]:
from transformers import LlamaTokenizer, LlamaTokenizerFast
tokenizer = LlamaTokenizerFast.from_pretrained('/data/home/wangys/model/llama2-13b-chat')
token_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in tqdm(prompt_opt['instruction'].to_list())]

In [1]:
from transformers import RobertaTokenizer
import pandas as pd
from tqdm.notebook import tqdm
prompt_opt = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/ant_buy_train_output.json')
# prompt_opt = CMS_train_few_output
prompt_opt.columns = ['instruction','input','output']
# 加载 RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('/data/home/wangys/roberta-base')
# tokenizer = LlamaTokenizerFast.from_pretrained('/data/home/wangys/model/llama2-13b-chat')

# 给定的列表
# text_list = [
#     "This is the first sentence.",
#     "Here is another sentence.",
#     "Yet another example sentence."
# ]

# 统计每个元素的 token 长度
token_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in tqdm(prompt_opt['instruction'].to_list())]

print("Token Lengths:", max(token_lengths))

  0%|          | 0/7405 [00:00<?, ?it/s]

Token Lengths: 4417


In [5]:
import numpy as np
np.sum(np.array(token_lengths)>4096)

19

### semi-text-w

In [290]:
np.histogram(token_lengths)

(array([  11,  127,  824, 2427, 3342, 2321,  963,  319,   60,    9]),
 array([1606., 1766., 1926., 2086., 2246., 2406., 2566., 2726., 2886.,
        3046., 3206.]))

In [240]:
np.sum(np.array(token_lengths)>4700)

67

## for semi_text_w,we add the labeled few-shot data, and the self-augmented and self-aligned data

In [84]:
semi_text_w_train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/semi-text-c/train_add.csv',index_col=0)

In [83]:
# semi_text_w_train
dict_ltable = np.load('/data/home/wangys/LLM_ER/process_data/semi-text-c/dict_ltable.npy',allow_pickle=True).item()
dict_rtable = np.load('/data/home/wangys/LLM_ER/process_data/semi-text-c/dict_ltable.npy',allow_pickle=True).item()
semi_text_w = np.load('/data/home/wangys/LLM_ER/process_data/semi-text-c/semi-text-c.npy',allow_pickle=True)

In [86]:
semi_text_w_train.columns = ['ltable_id','rtable_id','label']

In [87]:
query_dict = {}
for index,row in semi_text_w_train.iterrows():
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    left = str(dict_ltable[ltable_id])
    if(not query_dict.__contains__(left)):
        query_dict[left] = {}
        query_dict[left]['pos'] = []
        query_dict[left]['neg'] = []
    if(row['label']==0):
        tmp = query_dict[left]['neg']
        tmp.append(str(dict_rtable[rtable_id]))
        query_dict[left]['neg'] = tmp
    else:
        tmp = query_dict[left]['pos']
        tmp.append(str(dict_rtable[rtable_id]))
        query_dict[left]['pos'] = tmp

In [88]:
a = pd.DataFrame(query_dict).T
a['query'] = a.index
a = a.reset_index(drop=True)
a

,pos,neg,query
0,"[{'/mpn': '[u16gx4m2a2666c16r, cmu16gx4m2a266...",[],"{'/productID': '[csrlp2664]', '/mpn': '[cmk64g..."
1,"[{'/productID': '[csr2616rl]', '/mpn': '[cmu16...",[],"{'/productID': '[csrlp2664]', '/mpn': '[cmk64g..."
2,[],"[{'/productID': '[843591072816]', 'category': ...","{'/productID': '[csrlp2664]', '/mpn': '[cmk64g..."
3,"[{'/mpn': '[hpeh0146fawjb]', 'brand': '""HP Ent...","[{'/sku': '[3ra3851aa5pack]', '/mpn': '[3ra385...","{'/sku': '[dh0072balwl2pack]', '/mpn': '[dh007..."
4,"[{'/sku': '[dg0146famwl5pack]', '/mpn': '[dg01...","[{'/mpn': '[404712001, 404712001b]', '/gtin13...","{'/sku': '[dh0072balwl2pack]', '/mpn': '[dh007..."
...,...,...,...
3210,[],"[{'/mpn': '[hp456896001]', 'brand': 'HP', 'cat...","{'/sku': '[43194300310pack]', '/mpn': '[431943..."
3211,"[{'/mpn': '[483403b21]', 'brand': 'Genuine HPE...","[{'/sku': '[500670b212pack]', '/mpn': '[500670...","{'/mpn': '[432806b21]', 'brand': 'HP', 'catego..."
3212,"[{'/sku': '[470065235]', 'brand': 'Hewlett Pac...","[{'/mpn': '[356816001]', 'brand': 'Hewlett-Pac...","{'/sku': '[453475b21]', '/mpn': '[453475b21]',..."
3213,[],"[{'/sku': '[507825b21]', '/mpn': '[507825b21]'...","{'/sku': '[453475b21]', '/mpn': '[453475b21]',..."


In [100]:
a_output = a[(a['pos'].astype(str)!='[]') & (a['neg'].astype(str)!='[]')]

In [102]:
df = a_output
# 打开一个文件以写入
import json
with open('/data/home/wangys/LLM_ER/blocking/semi-text-c.json', 'w', encoding='utf-8') as file:
    for index, row in tqdm(df.iterrows()):
        # 对于每行，创建一个字典
        data = {
            "query": str(row["query"]),
            "pos": row["pos"],
            "neg": row["neg"]
        }

        # 将字典转换为JSON字符串，并写入文件
        file.write(json.dumps(data, ensure_ascii=False))
        file.write('\n')  # 每个JSON对象后添加换行符

print("/data/home/wangys/LLM_ER/blocking/semi-text-c.jsonl")

0it [00:00, ?it/s]

/data/home/wangys/LLM_ER/blocking/semi-text-c.jsonl


In [37]:
semi_text_w_left = pd.read_json('/data/home/wangys/LLM_ER/process_data/semi-text-w/semi-text-w/left.json')
entities = []
file_path = "/data/home/wangys/LLM_ER/process_data/semi-text-w/semi-text-w/right.txt"
with open(file_path, "r",encoding='utf-8') as rd:
    lines = rd.readlines()
    for line in tqdm(lines, desc="read text entity..."):
        text = line.strip()
        entities.append(text)
semi_text_w_right = entities

read text entity...:   0%|          | 0/9234 [00:00<?, ?it/s]

In [43]:
len(pd.Series(semi_text_w_right).unique())

1984

In [221]:
max_length = 4096
semi_text_w_train_gt = semi_text_w_train[semi_text_w_train['label']==1]
def Prompt_Generalize_semi_text_w(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s\n\nTake these examples as reference:\n\n' % (json.dumps(temp_dict),json.dumps(str(row['l_entity'])),json.dumps(str(row['r_entity'])))
    select_df = semi_text_w_train[(semi_text_w_train['ltable_id']==ltable_id) | (semi_text_w_train['rtable_id']==rtable_id)] ## 相同元素
    select_df = select_df[~((select_df['ltable_id']==ltable_id) & (select_df['rtable_id']==rtable_id))] ## 排除自己
    select_df_pos = select_df[select_df['label']==1]
    select_df_neg = select_df[select_df['label']==0]
    all_neg = semi_text_w_train[semi_text_w_train['label']==0]
    # if(len(select_df_pos)>=3): ## Contain pos
    #     for index,row_ in select_df_pos.sample(n=4,replace=False).iterrows():
    #         print(index)
    #         RAG_dict = {}
    #         RAG_dict['Entity_1'] = str(row_['l_entity'])
    #         RAG_dict['Entity_2'] = str(row_['r_entity'])
    #         RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
    #         RAG_text += '%s\n\n' % json.dumps(RAG_dict)
    # else:
    if(len(select_df_neg)>=3):
        df = pd.concat([semi_text_w_train_gt.sample(n=3,replace=False),select_df_neg.sample(n=3,replace=False)]).sample(frac=1)
    else:
        df = pd.concat([semi_text_w_train_gt.sample(n=3,replace=False),select_df_neg.sample(frac=1,replace=False),all_neg.sample(n=3-len(select_df_neg),replace=False)]).sample(frac=1)
    RAG_text_bck = RAG_text
    for index,row_ in df.iterrows():
        
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(row_['l_entity'])
        RAG_dict['Entity_2'] = str(row_['r_entity'])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text_bck += '%s\n\n' % str(RAG_dict)
        if(len(tokenizer.encode(RAG_text_bck, add_special_tokens=True))<max_length):
            RAG_text += '%s\n\n' % str(RAG_dict)
        else:
            break
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
semi_text_w_output = semi_text_w_train.progress_apply(Prompt_Generalize_semi_text_w,axis=1,result_type='expand')    
# semi_text_w_output_test = semi_text_w_test.progress_apply(Prompt_Generalize_semi_text_w,axis=1,result_type='expand')           
       
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 4258/4258 [00:51<00:00, 83.42it/s]


In [175]:
# print(semi_text_w_output.iloc[0,0])
# semi_text_w_output.columns = ['instruction','input','output']
semi_text_w_output_test.columns = ['instruction','input','output']

## We try RAG on semi-text-w

In [177]:
semi_text_c_train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/semi-text-c/train_add.csv',index_col=0)

In [199]:
semi_text_c_test = pd.read_csv('/data/home/wangys/LLM_ER/process_data/semi-text-c/semi-text-c/test.csv',header=None)

In [183]:
semi_text_c_ltable_dict = np.load('/data/home/wangys/LLM_ER/process_data/semi-text-c/dict_ltable.npy',allow_pickle=True).item()
semi_text_c_rtable_dict = np.load('/data/home/wangys/LLM_ER/process_data/semi-text-c/dict_rtable.npy',allow_pickle=True).item()

In [188]:
# semi_text_c_train.columns = semi_text_w_train.columns[:3]
semi_text_c_train['l_entity'] = semi_text_c_train['ltable_id'].map(semi_text_c_ltable_dict)
semi_text_c_train['r_entity'] = semi_text_c_train['rtable_id'].map(semi_text_c_rtable_dict)

In [201]:
semi_text_c_test.columns = semi_text_w_train.columns[:3]
semi_text_c_test['l_entity'] = semi_text_c_test['ltable_id'].map(semi_text_c_ltable_dict)
semi_text_c_test['r_entity'] = semi_text_c_test['rtable_id'].map(semi_text_c_rtable_dict)

In [194]:
semi_text_c_train_ind = semi_text_c_train.iloc[:,-2:].astype(str).drop_duplicates().index
semi_text_c_train = semi_text_c_train.loc[semi_text_c_train_ind]

In [227]:
semi_text_c_train['index'] = semi_text_c_train.index

In [232]:
semi_text_c_train_gt = semi_text_c_train[semi_text_c_train['label']==1]
def Prompt_Generalize_semi_text_c(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s\n\nTake these examples as reference:\n\n' % (json.dumps(temp_dict),json.dumps(str(row['l_entity'])),json.dumps(str(row['r_entity'])))
    select_df = semi_text_c_train[(semi_text_c_train['ltable_id']==ltable_id) | (semi_text_c_train['rtable_id']==rtable_id)] ## 相同元素
    select_df = select_df[~((select_df['ltable_id']==ltable_id) & (select_df['rtable_id']==rtable_id))] ## 排除自己
    select_df_pos = select_df[select_df['label']==1]
    select_df_neg = select_df[select_df['label']==0]
    all_neg = semi_text_c_train[semi_text_c_train['label']==0]
    # if(len(select_df_pos)>=3): ## Contain pos
    #     for index,row_ in select_df_pos.sample(n=4,replace=False).iterrows():
    #         print(index)
    #         RAG_dict = {}
    #         RAG_dict['Entity_1'] = str(row_['l_entity'])
    #         RAG_dict['Entity_2'] = str(row_['r_entity'])
    #         RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
    #         RAG_text += '%s\n\n' % json.dumps(RAG_dict)
    # else:
    if(len(select_df_neg)>=3):
        df = pd.concat([semi_text_c_train_gt.sample(n=3,replace=False),select_df_neg.sample(n=3,replace=False)]).sample(frac=1)
    else:
        df = pd.concat([semi_text_c_train_gt.sample(n=3,replace=False),select_df_neg.sample(frac=1,replace=False),all_neg.sample(n=3-len(select_df_neg),replace=False)]).sample(frac=1)
    RAG_text_bck = RAG_text
    for index,row_ in df.iterrows():
        
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(row_['l_entity'])
        RAG_dict['Entity_2'] = str(row_['r_entity'])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text_bck += '%s\n\n' % str(RAG_dict)
        RAG_length = len(tokenizer.encode(RAG_text_bck, add_special_tokens=True))
        if(<max_length):
            RAG_text += '%s\n\n' % str(RAG_dict)
        else:
            # print(row['index'])
            break
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
# semi_text_c_output = semi_text_c_train.progress_apply(Prompt_Generalize_semi_text_c,axis=1,result_type='expand')    
semi_text_c_output_test = semi_text_c_test.progress_apply(Prompt_Generalize_semi_text_c,axis=1,result_type='expand')           
       
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 4179/4179 [02:19<00:00, 30.04it/s]


In [233]:
semi_text_c_output.columns = ['instruction','input','output']
semi_text_c_output_test.columns = ['instruction','input','output']

In [242]:
## Walmart-Amazon from here
walmart_amazon_train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/walmart-amazon/train.csv',index_col=0)

In [251]:
walmart_amazon_test = pd.read_csv('/data/home/wangys/LLM_ER/process_data/walmart-amazon/test_align.csv',index_col=0)

In [254]:
walmart_amazon_train_gt = walmart_amazon_train[walmart_amazon_train['label']==1]
def Prompt_Generalize_walmart_amazon(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s\n\nTake these examples as reference:\n\n' % (json.dumps(temp_dict),json.dumps(str(row['l_entity'])),json.dumps(str(row['r_entity'])))
    select_df = walmart_amazon_train[(walmart_amazon_train['ltable_id']==ltable_id) | (walmart_amazon_train['rtable_id']==rtable_id)] ## 相同元素
    select_df = select_df[~((select_df['ltable_id']==ltable_id) & (select_df['rtable_id']==rtable_id))] ## 排除自己
    select_df_pos = select_df[select_df['label']==1]
    select_df_neg = select_df[select_df['label']==0]
    all_neg = walmart_amazon_train[walmart_amazon_train['label']==0]
    # if(len(select_df_pos)>=3): ## Contain pos
    #     for index,row_ in select_df_pos.sample(n=4,replace=False).iterrows():
    #         print(index)
    #         RAG_dict = {}
    #         RAG_dict['Entity_1'] = str(row_['l_entity'])
    #         RAG_dict['Entity_2'] = str(row_['r_entity'])
    #         RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
    #         RAG_text += '%s\n\n' % json.dumps(RAG_dict)
    # else:
    if(len(select_df_neg)>=4):
        df = pd.concat([walmart_amazon_train_gt.sample(n=3,replace=False),select_df_neg.sample(n=4,replace=False)]).sample(frac=1)
    else:
        df = pd.concat([walmart_amazon_train_gt.sample(n=4,replace=False),select_df_neg.sample(frac=1,replace=False),all_neg.sample(n=4-len(select_df_neg),replace=False)]).sample(frac=1)
    for index,row_ in df.iterrows():
        
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(row_['l_entity'])
        RAG_dict['Entity_2'] = str(row_['r_entity'])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        # RAG_text_bck += '%s\n\n' % str(RAG_dict)
        # RAG_length = len(tokenizer.encode(RAG_text_bck, add_special_tokens=True))
        RAG_text += '%s\n\n' % str(RAG_dict)
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
# walmart_amazon_train_output = walmart_amazon_train.progress_apply(Prompt_Generalize_walmart_amazon,axis=1,result_type='expand') 
walmart_amazon_test_output = walmart_amazon_test.progress_apply(Prompt_Generalize_walmart_amazon,axis=1,result_type='expand')    
# semi_text_c_output = semi_text_c_train.progress_apply(Prompt_Generalize_semi_text_c,axis=1,result_type='expand')    
# semi_text_c_output_test = semi_text_c_test.progress_apply(Prompt_Generalize_semi_text_c,axis=1,result_type='expand')  

pandas bar: 100%|██████████| 2049/2049 [00:05<00:00, 348.23it/s]


In [263]:
## wdc-all We try to concat multiple train files
wdc_all_train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/wdc-all/wdc_all/train.csv',index_col=0)

In [265]:
wdc_all_train_sample = pd.read_csv('/data/home/wangys/LLM_ER/process_data/wdc-all/train.csv',index_col=0)

In [284]:
wdc_all_test = pd.read_csv('/data/home/wangys/LLM_ER/process_data/wdc-all/wdc_all/dpo_test.csv',index_col=0)
wdc_all_test

,l_entity,r_entity,label,ltable_id,rtable_id,l_entity_diff,r_entity_diff
9038,"{'title': '""GoPro HDMI Cable""@en Cable AHDMC-3...","{\n ""title"": "" ""GoPro Headstrap Plus Quickcli...",0,9038,9038,"{\n ""title"": "" ""GoPro HDMI Cable""@en Cable AH...","{\n ""title"": "" ""GoPro Headstrap Plus Quickcli..."
9039,"{\n ""title"": ""Benq ZOWIE RL2455 24\"" Full HD ...","{\n ""title"": ""Zowie RL2455 E-Sports 24\"" Full...",1,9039,9039,"{'title': '""Benq ZOWIE RL2455 24"" Full HD TN G...","{\n ""title"": ""Zowie RL2455 E-Sports 24\"" Full..."
9040,"{'title': '"" Mens Nixon The Crew Watch A1186-0...","{\n ""title"": ""\"" Mens Nixon The Brigade Watch...",0,9040,9040,"{\n ""title"": "" Mens Nixon The Crew Watch A118...","{\n ""title"": "" Mens Nixon The Brigade Watch A..."
9041,{'title': 'Nike Metcon 2 - Black/White/Wolf Gr...,"{'title': '""Nike Metcon DSX Flyknit - Wolf Gre...",0,9041,9041,"{'title': '""Nike Metcon 2 - Black/White/Wolf G...","{'title': '""Nike Metcon DSX Flyknit - Wolf Gre..."
9042,"{'title': '""PowerLine HD Day Night Cloud Camer...","{'title': 'HD DVR Surveillance System, 16 Infr...",0,9042,9042,{'title': 'PowerLine HD Day Night Cloud Camera...,"{'title': 'HD DVR Surveillance System, 16 Infr..."
...,...,...,...,...,...,...,...
13431,"{\n ""title"": ""ASUS GeForce GTX 1070 TURBO 8GB...","{\n ""title"": "" ""ASUS NVIDIA GeForce GTX 1070 ...",1,13431,13431,"{\n ""title"": ""ASUS GeForce GTX 1070 TURBO 8GB...","{'title': '""ASUS NVIDIA GeForce GTX 1070 Turbo..."
13432,"{'title': '"" Seiko Men\'s Solar Powered Watch""...","{'title': ""Seiko Astron Men's GPS Titanium Chr...",0,13432,13432,"{'title': '"" Seiko Men\'s Solar Powered Watch""...","{'title': '"" Seiko Astron Men\'s GPS Titanium ..."
13433,{'title': 'Fujifilm Instax Mini Film Twin Pack...,"{'title': '""Fujifilm Instax Mini 8 Instant Fil...",0,13433,13433,{'title': 'Fujifilm Instax Mini Film Twin Pack...,{'title': 'Fujifilm Instax Mini 8 Instant Film...
13434,"{'title': '""Skagen SKW6237 Holst mens watch""'}","{\n ""title"": "" "" Mens Skagen Signatur Regulat...",0,13434,13434,"{\n ""title"": "" ""Skagen SKW6237 Holst mens wat...","{\n ""title"": "" "" Mens Skagen Signatur Regulat..."


In [276]:
wdc_all_train = pd.concat([wdc_all_train,wdc_all_train_sample]).reset_index(drop=True)

In [294]:
wdc_all_train.to_csv('/data/home/wangys/LLM_ER/process_data/wdc-all/train_unique.csv')

In [286]:
wdc_all_train_gt = wdc_all_train[wdc_all_train['label']==1]
def Prompt_Generalize_wdc_all(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s\n\nTake these examples as reference:\n\n' % (json.dumps(temp_dict),json.dumps(str(row['l_entity'])),json.dumps(str(row['r_entity'])))
    select_df = wdc_all_train[(wdc_all_train['ltable_id']==ltable_id) | (wdc_all_train['rtable_id']==rtable_id)] ## 相同元素
    select_df = select_df[~((select_df['ltable_id']==ltable_id) & (select_df['rtable_id']==rtable_id))] ## 排除自己
    select_df_pos = select_df[select_df['label']==1]
    select_df_neg = select_df[select_df['label']==0]
    all_neg = wdc_all_train[wdc_all_train['label']==0]
    # if(len(select_df_pos)>=3): ## Contain pos
    #     for index,row_ in select_df_pos.sample(n=4,replace=False).iterrows():
    #         print(index)
    #         RAG_dict = {}
    #         RAG_dict['Entity_1'] = str(row_['l_entity'])
    #         RAG_dict['Entity_2'] = str(row_['r_entity'])
    #         RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
    #         RAG_text += '%s\n\n' % json.dumps(RAG_dict)
    # else:
    if(len(select_df_neg)>=4):
        df = pd.concat([wdc_all_train_gt.sample(n=4,replace=False),select_df_neg.sample(n=4,replace=False)]).sample(frac=1)
    else:
        df = pd.concat([wdc_all_train_gt.sample(n=4,replace=False),select_df_neg.sample(frac=1,replace=False),all_neg.sample(n=4-len(select_df_neg),replace=False)]).sample(frac=1)
    for index,row_ in df.iterrows():
        
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(row_['l_entity'])
        RAG_dict['Entity_2'] = str(row_['r_entity'])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        # RAG_text_bck += '%s\n\n' % str(RAG_dict)
        # RAG_length = len(tokenizer.encode(RAG_text_bck, add_special_tokens=True))
        RAG_text += '%s\n\n' % str(RAG_dict)
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
# wdc_all_train_output = wdc_all_train.progress_apply(Prompt_Generalize_wdc_all,axis=1,result_type='expand')
wdc_all_test_output = wdc_all_test.progress_apply(Prompt_Generalize_wdc_all,axis=1,result_type='expand')

# walmart_amazon_train_output = walmart_amazon_train.progress_apply(Prompt_Generalize_walmart_amazon,axis=1,result_type='expand') 
# walmart_amazon_test_output = walmart_amazon_test.progress_apply(Prompt_Generalize_walmart_amazon,axis=1,result_type='expand')    
# semi_text_c_output = semi_text_c_train.progress_apply(Prompt_Generalize_semi_text_c,axis=1,result_type='expand')    
# semi_text_c_output_test = semi_text_c_test.progress_apply(Prompt_Generalize_semi_text_c,axis=1,result_type='expand')  

pandas bar:   0%|          | 0/4398 [00:00<?, ?it/s]

pandas bar: 100%|██████████| 4398/4398 [00:14<00:00, 310.88it/s]


In [288]:
wdc_all_train_output.columns = ['instruction','input','output']
wdc_all_test_output.columns = ['instruction','input','output']

In [ ]:
## Ant_Buy

In [295]:
ant_buy_train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/ant-buy/train.csv',index_col=0)

In [297]:
ant_buy_test = pd.read_csv('/data/home/wangys/LLM_ER/process_data/ant-buy/exp_data/dpo_test.csv',index_col=0)

In [304]:
ant_buy_train_gt = ant_buy_train[ant_buy_train['label']==1]
def Prompt_Generalize_ant_buy(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s\n\nTake these examples as reference:\n\n' % (json.dumps(temp_dict),json.dumps(str(row['l_entity'])),json.dumps(str(row['r_entity'])))
    select_df = ant_buy_train[(ant_buy_train['ltable_id']==ltable_id) | (ant_buy_train['rtable_id']==rtable_id)] ## 相同元素
    select_df = select_df[~((select_df['ltable_id']==ltable_id) & (select_df['rtable_id']==rtable_id))] ## 排除自己
    select_df_pos = select_df[select_df['label']==1]
    select_df_neg = select_df[select_df['label']==0]
    all_neg = ant_buy_train[ant_buy_train['label']==0]
    # if(len(select_df_pos)>=3): ## Contain pos
    #     for index,row_ in select_df_pos.sample(n=4,replace=False).iterrows():
    #         print(index)
    #         RAG_dict = {}
    #         RAG_dict['Entity_1'] = str(row_['l_entity'])
    #         RAG_dict['Entity_2'] = str(row_['r_entity'])
    #         RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
    #         RAG_text += '%s\n\n' % json.dumps(RAG_dict)
    # else:
    if(len(select_df_neg)>=4):
        df = pd.concat([ant_buy_train_gt.sample(n=4,replace=False),select_df_neg.sample(n=4,replace=False)]).sample(frac=1)
    else:
        df = pd.concat([ant_buy_train_gt.sample(n=4,replace=False),select_df_neg.sample(frac=1,replace=False),all_neg.sample(n=4-len(select_df_neg),replace=False)]).sample(frac=1)
    for index,row_ in df.iterrows():
        
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(row_['l_entity'])
        RAG_dict['Entity_2'] = str(row_['r_entity'])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        # RAG_text_bck += '%s\n\n' % str(RAG_dict)
        # RAG_length = len(tokenizer.encode(RAG_text_bck, add_special_tokens=True))
        RAG_text += '%s\n\n' % str(RAG_dict)
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
# ant_buy_train_output = ant_buy_train.progress_apply(Prompt_Generalize_ant_buy,axis=1,result_type='expand')
ant_buy_test_output = ant_buy_test.progress_apply(Prompt_Generalize_ant_buy,axis=1,result_type='expand')
# wdc_all_test_output = wdc_all_test.progress_apply(Prompt_Generalize_wdc_all,axis=1,result_type='expand')

pandas bar:   0%|          | 0/1916 [00:00<?, ?it/s]

pandas bar: 100%|██████████| 1916/1916 [00:05<00:00, 341.97it/s]


In [305]:
ant_buy_train_output.columns = ['instruction','input','output']
ant_buy_test_output.columns = ['instruction','input','output']

### Schema_Matching Dataset Process
### Use the same template as Entity Matching

In [538]:
CMS = pd.read_excel('/data/home/wangys/LLM_ER/process_data/SM/omop_cms_data.xlsx')

In [436]:
mimic = pd.read_excel('/data/home/wangys/LLM_ER/process_data/SM/omop_mimic_data.xlsx')

In [439]:
synthea = pd.read_excel('/data/home/wangys/LLM_ER/process_data/SM/omop_synthea_data.xlsx')

In [460]:
CMS = synthea

In [539]:
CMS['tableA'] = CMS['omop'] + '|' + CMS['des1'] 
CMS['tableB'] = CMS['table'] + '|' + CMS['des2'] 

In [540]:
CMS_tableA = list(CMS['tableA'].unique())
CMS_tableB = list(CMS['tableB'].unique())
CMS_tableA_dict = {}
CMS_tableB_dict = {}
for i in tqdm(range(len(CMS_tableA))):
    CMS_tableA_dict[CMS_tableA[i]] = i
for i in tqdm(range(len(CMS_tableB))):
    CMS_tableB_dict[CMS_tableB[i]] = i

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

In [541]:
CMS_tableA_dict_rev = {v: k for k, v in CMS_tableA_dict.items()}
CMS_tableB_dict_rev = {v: k for k, v in CMS_tableB_dict.items()}

In [542]:
# len(CMS['table'].unique())
CMS['tableA_id'] = CMS['tableA'].map(CMS_tableA_dict)
CMS['tableB_id'] = CMS['tableB'].map(CMS_tableB_dict)

In [465]:
CMS_shuffle = CMS.sample(frac=1)
CMS_train = CMS_shuffle.iloc[:int(len(CMS_shuffle)*0.8)]
CMS_test = CMS_shuffle.iloc[int(len(CMS_shuffle)*0.8):]

In [466]:
CMS_train_few = CMS_train.sample(frac=0.125)

In [467]:
CMS_train_few.to_csv('/data/home/wangys/LLM_ER/process_data/SM/synthea_train_few.csv')
CMS_train.to_csv('/data/home/wangys/LLM_ER/process_data/SM/synthea_train.csv')
CMS_test.to_csv('/data/home/wangys/LLM_ER/process_data/SM/synthea_test.csv')

In [619]:
CMS_train_few = pd.read_csv('/data/home/wangys/LLM_ER/process_data/SM/synthea_train_few.csv',index_col=0)
CMS_train = pd.read_csv('/data/home/wangys/LLM_ER/process_data/SM/synthea_train.csv',index_col=0)
CMS_test = pd.read_csv('/data/home/wangys/LLM_ER/process_data/SM/synthea_test.csv',index_col=0)

In [568]:
# CMS_train_few
## Train Sentence_BERT
### For Few-shot 
query_list = []
# for index,row in CMS_train_few.iterrows():
#     query = row['tableA']
#     pos = list(CMS[(CMS['tableA']==query) & (CMS['label']==1)]['tableB'].unique())
#     neg = list(CMS_train_few[(CMS_train_few['tableA']==query)& (CMS_train_few['label']==0)]['tableB'].unique())
#     if(pos!=[]):
#         query_list.append([query,pos,neg])
### For All
# for index,row in tqdm(CMS_train.iterrows()):
for query in tqdm(CMS_train['tableA'].unique()):
    # query = row['tableA']
    pos = list(CMS_train[(CMS_train['tableA']==query) & (CMS['label']==1)]['tableB'].unique()) ## List All Positive with same query
    neg = list(CMS_train[(CMS_train['tableA']==query)& (CMS_train['label']==0)]['tableB'].unique())
    # if(pos!=[]):
    query_list.append([query,pos,neg])

  0%|          | 0/267 [00:00<?, ?it/s]

/tmp/ipykernel_2007539/2076948446.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pos = list(CMS_train[(CMS_train['tableA']==query) & (CMS['label']==1)]['tableB'].unique()) ## List All Positive with same query


In [ ]:
CMS_pos = CMS[CMS['label']==1]
# CMS_pos = CMS_train[CMS_train['label']==1]
for i in range(len(CMS_tableB_dict)):
# for i in CMS_test[CMS_test['label']==1]['tableA_id'].unique():
    length = len(CMS_pos[(CMS_pos['tableB_id']==i)])
    # if(length>1):
    print(length,i)

In [571]:
query_list_pd = pd.DataFrame(query_list)
query_list_pd.astype(str).drop_duplicates()
# query_list_pd[query_list_pd[1].astype(str)!='[]']

,0,1,2
0,death-person_id|the death domain contains the ...,['outpatientclaims-desynpuf_id|outpatientclaim...,['outpatientclaims-clm_id|outpatientclaims per...
1,procedure_occurrence-procedure_occurrence_id|t...,[],['outpatientclaims-admtng_icd9_dgns_cd|outpati...
2,note-note_date|the note table captures unstruc...,[],['beneficiarysummary-bene_esrd_ind|beneficiary...
3,person-birth_datetime|the person domain contai...,[],['carrierclaims-prf_physn_npi|carrierclaims pe...
4,device_exposure-device_source_concept_id|the d...,[],['outpatientclaims-at_physn_npi|outpatientclai...
...,...,...,...
262,measurement-measurement_time|the measurement t...,[],['outpatientclaims-icd9_prcdr_cd|outpatientcla...
263,measurement-measurement_source_concept_id|the ...,[],['beneficiarysummary-sp_chf|beneficiarysummary...
264,payer_plan_period-plan_source_value|the payer_...,['beneficiarysummary-bene_hmo_cvrage_tot_mons|...,['outpatientclaims-desynpuf_id|outpatientclaim...
265,measurement-operator_concept_id|the measuremen...,[],['inpatientclaims-hcpcs_cd|inpatientclaims per...


In [620]:
CMS_train_few['ltable_id'] = CMS_train_few['tableA_id']
CMS_train_few['rtable_id'] = CMS_train_few['tableB_id']
CMS_train['ltable_id'] = CMS_train['tableA_id']
CMS_train['rtable_id'] = CMS_train['tableB_id']

In [621]:
CMS_test['ltable_id'] = CMS_test['tableA_id']
CMS_test['rtable_id'] = CMS_test['tableB_id']

In [591]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# CMS_train_few_gt = CMS_train_few[CMS_train_few['label']==1]
CMS_train_few_gt = CMS_train[CMS_train['label']==1]
CMS_train_few = CMS_train
def Prompt_Generalize_CMS(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    col1_dict = {}
    col1_dict['column_name'] = row['tableA'].split('|')[0]
    col1_dict['description'] = row['tableA'].split('|')[1]
    col2_dict = {}
    col2_dict['column_name'] = row['tableB'].split('|')[0]
    col2_dict['description'] = row['tableB'].split('|')[1]
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two columns in tables refer to the same or different schema. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Column 1 and Column 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nColumn 1:%s\n\nColumn 2:%s\n\nTake these examples as reference:\n\n' % (json.dumps(temp_dict),json.dumps(str(col1_dict)),json.dumps(str(col2_dict)))
    select_df = CMS_train_few_gt[(CMS_train_few_gt['ltable_id']==ltable_id) | (CMS_train_few_gt['rtable_id']==rtable_id)] ## 相同元素
    select_df = select_df[~((select_df['ltable_id']==ltable_id) & (select_df['rtable_id']==rtable_id))] ## 排除自己
    select_df_pos = select_df[select_df['label']==1]
    select_df_neg = select_df[select_df['label']==0]
    all_neg = CMS_train_few[CMS_train_few['label']==0]
    # if(len(select_df_pos)>=3): ## Contain pos
    #     for index,row_ in select_df_pos.sample(n=4,replace=False).iterrows():
    #         print(index)
    #         RAG_dict = {}
    #         RAG_dict['Entity_1'] = str(row_['l_entity'])
    #         RAG_dict['Entity_2'] = str(row_['r_entity'])
    #         RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
    #         RAG_text += '%s\n\n' % json.dumps(RAG_dict)
    # else:
    if(len(select_df_neg)>=4):
        df = pd.concat([CMS_train_few_gt.sample(n=2,replace=False),select_df_neg.sample(n=4,replace=False)]).sample(frac=1)
    else:
        df = pd.concat([CMS_train_few_gt.sample(n=2,replace=False),select_df_neg.sample(frac=1,replace=False),all_neg.sample(n=4-len(select_df_neg),replace=False)]).sample(frac=1)
    for index,row_ in df.iterrows():
        col1_dict = {}
        col1_dict['column_name'] = row_['tableA'].split('|')[0]
        col1_dict['description'] = row_['tableA'].split('|')[1]
        col2_dict = {}
        col2_dict['column_name'] = row_['tableB'].split('|')[0]
        col2_dict['description'] = row_['tableB'].split('|')[1]
        ltable_id = row['ltable_id']
        rtable_id = row['rtable_id']
        RAG_dict = {}
        RAG_dict['Column 1'] = str(col1_dict)
        RAG_dict['Column 2'] = str(col2_dict)
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        # RAG_text_bck += '%s\n\n' % str(RAG_dict)
        # RAG_length = len(tokenizer.encode(RAG_text_bck, add_special_tokens=True))
        RAG_text += '%s\n\n' % str(RAG_dict)
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    # return text + RAG_text, '' , str(output_dict)
    return text + RAG_text, '' , str(output_dict), row['ltable_id'], row['rtable_id']
# CMS_train_few_output = CMS_train_few.progress_apply(Prompt_Generalize_CMS,axis=1,result_type='expand')
# CMS_test_few_output = CMS_test.progress_apply(Prompt_Generalize_CMS,axis=1,result_type='expand')
CMS_train_output = CMS_train.parallel_apply(Prompt_Generalize_CMS,axis=1,result_type='expand')
CMS_test_output = CMS_test.progress_apply(Prompt_Generalize_CMS,axis=1,result_type='expand')


In [623]:
CMS_train_output_pos = CMS_train_output[CMS_train_output[2]=="{'Output': 'match'}"]
CMS_test_output_pos = CMS_test_output[CMS_test_output[2]=="{'Output': 'match'}"]
CMS_train_output_select = CMS_train_output[(CMS_train_output[3].isin(CMS_test_output_pos[3].unique())) | (CMS_train_output[4].isin(CMS_test_output_pos[4].unique()))]
CMS_train_output_select = pd.concat([CMS_train_output_select,CMS_train_output_pos]).drop_duplicates()

In [624]:
CMS_train_output_select

,0,1,2,3,4
27879,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},251,18
25394,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},228,86
370,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},3,37
19854,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},178,96
7674,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},69,15
...,...,...,...,...,...
8418,You are an expert in detecting if two columns ...,,{'Output': 'match'},75,93
18664,You are an expert in detecting if two columns ...,,{'Output': 'match'},168,16
2687,You are an expert in detecting if two columns ...,,{'Output': 'match'},24,23
19252,You are an expert in detecting if two columns ...,,{'Output': 'match'},173,49


In [508]:
CMS_train_few_output.columns = ['instruction','input','output']
CMS_test_few_output.columns = ['instruction','input','output']

In [625]:
CMS_train_output_select.columns = ['instruction','input','output','ltable_id','rtable_id']
CMS_test_output.columns = ['instruction','input','output','ltable_id','rtable_id']

In [629]:
df = CMS_test_output.iloc[:,:3]
json.dump(df.to_dict(orient='records'), open('/data/home/wangys/LLaMA-Factory-main/data/MoE/SM/synthea_test_output.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [628]:
import hashlib


def encrypt(fpath: str, algorithm: str) -> str:
    with open(fpath, 'rb') as f:
        return hashlib.new(algorithm, f.read()).hexdigest()


if __name__ == '__main__':
    for algorithm in ('md5', 'sha1', 'sha256'):
        hexdigest = encrypt('/data/home/wangys/LLaMA-Factory-main/data/MoE/SM/synthea_train_output.json', algorithm)
        print(f'{algorithm}: {hexdigest}')

md5: 39d1f0f652a90a322cc5de4a62b29eb3
sha1: e7a9f54bb67d426530703c5937ecb7dfbd591378
sha256: ee55372343cd7c3e8d92fd07ab45a74bc62a0a34dc4715f39e4c108d970ff6e8


In [17]:
### Merge and Shuffle
amazon_google = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/amazon-google-train.json')
ant_buy = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/ant_buy_train_output.json')
semi_text_c = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/semi-text-c-train-MoE.json')
semi_text_w = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/semi-text-w-train-MoE.json')
walmart_amazon = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/walmart_amazon_train_output.json')
wdc_all = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/wdc_all_train_output.json')
hospital_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/hospital-train-MoE.json')
beer_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/beer-train-MoE.json')
rayyan_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/rayyan-train-MoE.json')
RE = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/RE/RE-train_t=4.json')
CMS_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/SM/CMS_train_output.json')
mimic_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/SM/mimic_train_output.json')
synthea_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/SM/synthea_train_output.json')
walmart_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/DI/walmart_train_output_wide.json')
amazon_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/DI/amazon_train_output_wide.json')
restaurant_train = pd.read_json('/data/home/wangys/LLaMA-Factory-main/data/MoE/DI/restaurant_train_output_wide.json')

In [18]:
# merge_df_input = [amazon_google,ant_buy,semi_text_c,semi_text_w,walmart_amazon,wdc_all,CMS_train,mimic_train,synthea_train]
merge_df_input = [RE,walmart_train,amazon_train,restaurant_train]

In [19]:
import pandas as pd


def merge_dataframes(dataframes):
    merged_dataframes = []

    for i, df in enumerate(dataframes):
        # 创建一个空的DataFrame用于累积合并的结果
        cumulative_merge = pd.DataFrame()

        for j, other_df in enumerate(dataframes):
            if i != j:
                # 根据output列中的元素种类执行不同的合并策略
                if other_df['output'].nunique() == 2:
                    # 执行原始策略
                    dismatch_rows = other_df[other_df['output'].str.contains('dismatch', na=False)]
                    non_dismatch_rows = other_df[~other_df['output'].str.contains('dismatch', na=False)]

                    selected_dismatch = dismatch_rows.sample(n=min(70, len(dismatch_rows)), replace=True)
                    selected_non_dismatch = non_dismatch_rows.sample(n=min(210, len(non_dismatch_rows)), replace=True)

                    merge_result = pd.concat([selected_dismatch, selected_non_dismatch])
                else:
                    # 随机选择150行数据
                    selected_rows = other_df.sample(n=min(150, len(other_df)), replace=True)
                    merge_result = selected_rows

                # 累积合并结果
                cumulative_merge = pd.concat([cumulative_merge, merge_result])

        # 将累积的合并结果与原始DataFrame合并
        final_merged_df = pd.concat([df, cumulative_merge])

        # 将最终合并的DataFrame添加到结果列表中
        merged_dataframes.append(final_merged_df)

    return merged_dataframes

merge_df_output = merge_dataframes(merge_df_input)

In [ ]:
def merge_dataframes(dataframes):
    merged_dataframes = []

    for df in dataframes:
        df.columns =['instruction','input','output']
        # 检查output列中元素的种类数量
        if df['output'].nunique() == 2:
            # 如果只有两种元素，执行原始策略
            dismatch_rows = df[df['output'].str.contains('dismatch', na=False)]
            non_dismatch_rows = df[~df['output'].str.contains('dismatch', na=False)]

            selected_dismatch = dismatch_rows.sample(n=min(80, len(dismatch_rows)), replace=True)
            selected_non_dismatch = non_dismatch_rows.sample(n=min(240, len(non_dismatch_rows)), replace=True)

            merged_df = pd.concat([df, selected_dismatch, selected_non_dismatch])
        else:
            # 如果有多于两种元素，随机选择150行数据
            selected_rows = df.sample(n=min(100, len(df)), replace=True)
            merged_df = pd.concat([df, selected_rows])

        # 将合并后的数据添加到新列表中
        merged_dataframes.append(merged_df)

    return merged_dataframes

In [13]:
def find_var_name(obj):
    for var_name in globals():
        if globals()[var_name] is obj:
            return var_name
    return None

In [22]:
def SHA1_cal(path):
    hexdigest = encrypt(path, 'sha1')
    return hexdigest
# SHA1_cal('/data/home/wangys/LLaMA-Factory-main/data/MoE/ER/ant_buy_train_output.json')

In [23]:
import json
dataset_info = {}
for i in range(len(merge_df_output)):
# for i in [6,7,8]:
    df = merge_df_output[i]
    name = find_var_name(merge_df_input[i])
    print(name)
    path_save = '/data/home/wangys/LLaMA-Factory-main/data/MoE/add/%s.json' % name
    json.dump(df.to_dict(orient='records'), open(path_save, 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    name += '-MoE-Add'
    dataset_info[name] = {}
    dataset_info[name]["file_name"] = path_save.replace('/data/home/wangys/LLaMA-Factory-main/data/','')
    dataset_info[name]["file_sha1"] = SHA1_cal(path_save)
    dataset_info[name]["columns"] = {
        "prompt": "instruction",
        "query": "input",
        "response": "output"
    }
    

RE
walmart_train
amazon_train
restaurant_train


In [30]:
merge_df_output[3]

,instruction,input,output
0,You are an expert in attribute extraction from...,,"{""city"": ""` los angeles '""}"
1,You are an expert in attribute extraction from...,,"{""city"": ""` studio city '""}"
2,You are an expert in attribute extraction from...,,"{""city"": ""` bel air '""}"
3,You are an expert in attribute extraction from...,,"{""city"": ""` sherman oaks '""}"
4,You are an expert in attribute extraction from...,,"{""city"": ""` los angeles '""}"
...,...,...,...
4218,You are an expert in attribute extraction from...,,{'category': 'projection screens'}
2109,You are an expert in attribute extraction from...,,{'category': 'mp3 players'}
4278,You are an expert in attribute extraction from...,,{'brand': 'griffin technology'}
6509,You are an expert in attribute extraction from...,,{'category': 'memory cards'}


In [24]:
str(dataset_info).replace("'",'"')[1:-1]

'"RE-MoE-Add": {"file_name": "MoE/add/RE.json", "file_sha1": "b759b90cb04b77c53a72034fcf9c4960df549667", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "walmart_train-MoE-Add": {"file_name": "MoE/add/walmart_train.json", "file_sha1": "771a5cadb52ce4f85b111db204627195c9c1fe4f", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "amazon_train-MoE-Add": {"file_name": "MoE/add/amazon_train.json", "file_sha1": "db097ac843ef02e63f951cd4575e13eb3bd35582", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "restaurant_train-MoE-Add": {"file_name": "MoE/add/restaurant_train.json", "file_sha1": "e0db121115abc361504e3f9ebf3651556702cc29", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}'

In [2]:
# str(dataset_info).replace("'",'"')[1:-1]
task_list = list(dataset_info.keys())
task_list

NameError: name 'dataset_info' is not defined

In [27]:
["WANDB_MODE=disabled accelerate launch src/train_bash.py     --stage sft     --model_name_or_path Mistral-7B-Instruct-v0.2    --do_train     --finetuning_type lora     --dataset %s     --output_dir lora_weight/MoE/add/Mistral/%s --overwrite_output_dir     --lr_scheduler_type cosine     --num_train_epochs 15.0     --gradient_accumulation_steps 8     --per_device_eval_batch_size 8     --fp16     --template mistral     --lora_r 16 --logging_steps 5 --plot_loss  --lora_target all --save_steps 50 --use_unsloth --cutoff_len 2048" % (t,t) for t in task_list[1:]]

['WANDB_MODE=disabled accelerate launch src/train_bash.py     --stage sft     --model_name_or_path Mistral-7B-Instruct-v0.2    --do_train     --finetuning_type lora     --dataset walmart_train-MoE-Add     --output_dir lora_weight/MoE/add/Mistral/walmart_train-MoE-Add --overwrite_output_dir     --lr_scheduler_type cosine     --num_train_epochs 15.0     --gradient_accumulation_steps 8     --per_device_eval_batch_size 8     --fp16     --template mistral     --lora_r 16 --logging_steps 5 --plot_loss  --lora_target all --save_steps 50 --use_unsloth --cutoff_len 2048',
 'WANDB_MODE=disabled accelerate launch src/train_bash.py     --stage sft     --model_name_or_path Mistral-7B-Instruct-v0.2    --do_train     --finetuning_type lora     --dataset amazon_train-MoE-Add     --output_dir lora_weight/MoE/add/Mistral/amazon_train-MoE-Add --overwrite_output_dir     --lr_scheduler_type cosine     --num_train_epochs 15.0     --gradient_accumulation_steps 8     --per_device_eval_batch_size 8     --fp16 

In [ ]:
WANDB_MODE=disabled accelerate launch src/train_bash.py     --stage sft     --model_name_or_path Mistral-7B-Instruct-v0.2    --do_train     --finetuning_type lora     --dataset hospital_train-MoE-Add     --output_dir lora_weight/MoE/add/Mistral/hospital_train-MoE-Add --overwrite_output_dir     --lr_scheduler_type cosine     --num_train_epochs 15.0     --gradient_accumulation_steps 8     --per_device_eval_batch_size 8     --fp16     --template mistral     --lora_r 16 --logging_steps 5 --plot_loss  --lora_target all --save_steps 50 --use_unsloth --cutoff_len 4096 --quantization_bit 8

## From Here we Start Imputation

In [489]:
walmart_all = pd.read_csv('/data/home/wangys/LLM_ER/process_data/walmart-amazon-dirty/exp_data/tableA.csv').fillna('')